In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.preprocessing import timeseries_dataset_from_array

In [2]:
BTC_df = pd.read_csv("data/crypto/BTC_1min.csv")

In [3]:
BTC_df.info()
BTC_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17113 entries, 0 to 17112
Columns: 156 entries, Unnamed: 0 to asks_market_notional_14
dtypes: float64(154), int64(1), object(1)
memory usage: 20.4+ MB


,Unnamed: 0,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,...,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
0,0,2021-04-07 11:33:41.122161+00:00,55896.285,0.01,4.448599e+06,3.899402e+06,-8.945138e-08,-0.000065,-0.000073,-0.000154,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1176.72998,0.0
1,1,2021-04-07 11:34:41.122161+00:00,55948.685,1.43,1.243244e+06,3.606502e+06,-1.277957e-05,-0.000013,-0.000242,-0.000242,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
2,2,2021-04-07 11:35:41.122161+00:00,56013.785,0.01,3.069094e+06,1.575759e+06,-8.926374e-08,-0.000009,-0.000070,-0.000088,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
3,3,2021-04-07 11:36:41.122161+00:00,55903.575,7.17,1.220819e+06,1.321487e+06,-6.412827e-05,-0.000182,-0.000188,-0.000225,...,0.0,312.040009,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
4,4,2021-04-07 11:37:41.122161+00:00,55899.995,0.01,2.011287e+06,3.083005e+06,-8.944544e-08,-0.000129,-0.000130,-0.000143,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


In [4]:
BTC_df.shape

(17113, 156)

In [5]:
BTC_df.drop("Unnamed: 0", inplace = True, axis = 1)
BTC_df.drop("system_time", inplace = True, axis = 1)

In [6]:
data_length = BTC_df.shape[0]

train_length = round(0.8*data_length)
train_length

13690

In [7]:
mean_df = BTC_df[:train_length].mean()
std_df = BTC_df[:train_length].std()

normalized_df = (BTC_df - mean_df) / std_df

In [8]:
# Set window size
window_size = 30

# Windows for training dataset
windows_train = timeseries_dataset_from_array(normalized_df[:(train_length)], normalized_df["midpoint"][window_size:(train_length+window_size)], window_size, sequence_stride=1, sampling_rate=1,
    batch_size=64, shuffle=False, seed=None, start_index=None, end_index=None
)


# Windows for test dataset
windows_test = timeseries_dataset_from_array(normalized_df[train_length:(data_length-window_size)], normalized_df["midpoint"][train_length + window_size:], 30, sequence_stride=1, sampling_rate=1,
    batch_size=64, shuffle=False, seed=None, start_index=None, end_index=None
)

In [12]:
print(windows_train.element_spec)
print(windows_test.element_spec)

for x_batch, y_batch in windows_train.take(1):
    print("Input shape:", x_batch.shape)
    print("Target shape:", y_batch.shape)

(TensorSpec(shape=(None, None, 154), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))
(TensorSpec(shape=(None, None, 154), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))
Input shape: (64, 30, 154)
Target shape: (64,)


2025-03-24 10:58:25.993531: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
TensorSpec(shape=(None, None, 154), dtype=tf.float64, ...)

SyntaxError: positional argument follows keyword argument (3227378884.py, line 1)

In [14]:
model_dense = keras.models.Sequential([
    keras.layers.Input(shape=(30, 154)),  # Explicit input layer
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(1)
])

model_dense.compile(loss="mse", optimizer="adam", metrics=["mse"])
history = model_dense.fit(windows_train, epochs=10,verbose=1)

Epoch 1/10


2025-03-24 11:01:02.287973: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at reshape_op.h:65 : INVALID_ARGUMENT: Only one input size may be -1, not both 0 and 1


InvalidArgumentError: Graph execution error:

Detected at node sequential_1_1/flatten_1_1/Reshape defined at (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/runpy.py", line 198, in _run_module_as_main

  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/runpy.py", line 88, in _run_code

  File "/Users/sion/venvs/venv312/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/sion/venvs/venv312/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/sion/venvs/venv312/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/sion/venvs/venv312/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/Library/Frameworks/Python.framework/Versions/3.

In [ ]:
column_names_2 = ['system_time',
 'midpoint',
 'spread',
 'buys',
 'sells',
 'bids_distance_0',
 'bids_notional_0',
'asks_distance_0',
'asks_notional_0',
               ]

df0= BTC_df[column_names_2]
df0['BidPrice0'] = df0['midpoint']* (1+df0['bids_distance_0'])
df0['AskPrice0'] = df0['midpoint'] *(1+df0['asks_distance_0'])
hc2001_df_one = df0[['system_time',
 'midpoint',
 'spread',
 'BidPrice0',
 'AskPrice0',
 'bids_notional_0',
'asks_notional_0',]]
hc2001_df_one.columns = ['time','mid_price','spread', 'BidPrice1', 'AskPrice1', 'BidVolume1', 'AskVolume1']
hc2001_df_one

In [ ]:
#Volume Order Imbalance 
df = pd.concat([hc2001_df_one['BidVolume1'], hc2001_df_one['BidVolume1'].shift(1).fillna(0), hc2001_df_one["dBid_price"]], axis=1)
df.columns = ['bv', 'sbv', 'dbp']
df